In [0]:
!pip install optuna

In [0]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from scipy.stats import skew
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split

import lightgbm as lgb
import optuna
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

In [0]:
'''
データのロード
'''
import pickle

pickle_file = "./droped29_HousePrice_data_kernel.pickle"
with open(pickle_file, 'rb') as f:
    X_train = pickle.load(f)
    Y_train = pickle.load(f)
    X_test = pickle.load(f)
    train = pickle.load(f)
    test = pickle.load(f)
    Id = pickle.load(f)

In [0]:
# Y_train.hist(bins=30)
# for col in X_train:
#   if X_train[col].skew() > 0.6:
#     print(col)

In [0]:
# 欠損値を含むデータの補正
concated = pd.concat([X_train, Y_train], axis = 1)
_ = concated.dropna()
X_train_droped = _.drop('SalePrice', axis=1)
Y_train_droped = _.SalePrice

In [0]:
X_train_filled = X_train.fillna(X_train.mean())

In [0]:
X_train_filled = X_train.copy()
for col in X_train:
  X_train_filled[col].fillna(X_train_filled[col].mean(), inplace=True) 

In [0]:
'''
ハイパーパラメータの最適化
'''
def objective(trial):
    # hypyer param
    alpha = trial.suggest_uniform('alpha', 0.0001, 0.001)
 
    # callback
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, 'validation-error')
 
    # model
    model = Lasso(
        alpha = alpha
    )
    

    kf = KFold(5, shuffle=True, random_state=1).get_n_splits(X_train_filled)
    rmse = np.sqrt(-cross_val_score(model, X_train_filled,Y_train, scoring="neg_mean_squared_error", cv=kf, verbose=0))
    
    return rmse.mean()

In [8]:
# 学習の実行
# https://optuna.readthedocs.io/en/stable/reference/study.html#optuna.study.Study.optimize
study = optuna.create_study()
study.optimize(func=objective, # 実行する関数
               n_trials=200, # HPO試行回数
               timeout=None, # 与えられた秒数後に学習を中止します。default=None
               n_jobs=-1 # 並列実行するjob数
              )

[I 2019-05-23 03:08:32,839] Finished trial#0 resulted in value: 0.13642985220964207. Current best value is 0.13642985220964207 with parameters: {'alpha': 0.0003580792195684528}.
[I 2019-05-23 03:08:32,841] Finished trial#1 resulted in value: 0.13666328533044175. Current best value is 0.13642985220964207 with parameters: {'alpha': 0.0003580792195684528}.
[I 2019-05-23 03:08:33,665] Finished trial#3 resulted in value: 0.13667909935641087. Current best value is 0.13642985220964207 with parameters: {'alpha': 0.0003580792195684528}.
[I 2019-05-23 03:08:33,677] Finished trial#2 resulted in value: 0.13656056406394435. Current best value is 0.13642985220964207 with parameters: {'alpha': 0.0003580792195684528}.
[I 2019-05-23 03:08:34,668] Finished trial#4 resulted in value: 0.13671804239632313. Current best value is 0.13642985220964207 with parameters: {'alpha': 0.0003580792195684528}.
[I 2019-05-23 03:08:34,670] Finished trial#5 resulted in value: 0.136443884308737. Current best value is 0.136

KeyboardInterrupt: ignored

In [0]:
reg = Lasso(alpha=0.0004)

In [0]:
reg.fit(X_train_droped, Y_train_droped)

Lasso(alpha=0.0004, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)